In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats

import subprocess
### subprocess.run('export SPARK_LOCAL_IP=0.0.0.0', shell=True)

sc = pyspark.SparkContext(appName="IPTV-Anomaly-Detection-Conviva")
sqlContext = SQLContext(sc)

import h2o
from h2o.automl import H2OAutoML

# import subprocess
### subprocess.run('unset http_proxy', shell=True)

h2o.init(ip="localhost",port=54321)
#
#
#
internal_predict_files="local:///home/joci/notebooks/data/conviva1.csv"

#
# Join with Internal Curation Data in urltopredict staged folder
from pyspark.sql import functions as F
internaldata_df1=sqlContext.read.csv(internal_predict_files,header='true')\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
internaldata_df1.printSchema()
###
###
#
train_pd=internaldata_df1.toPandas()
test_pd=internaldata_df1.toPandas().head(300)                                                                        
train=h2o.H2OFrame(train_pd)
test=h2o.H2OFrame(test_pd)
#
##
###### TRAINING PROCESS ############
# RBIND "Merge" all of 3 internal ccrawl and phishtank with rbind .
# function merge() doesn't work if both H2O/dataframes have same variables
#
### train = train_hf1.rbind(train_hf2)
### test = test_hf1.rbind(test_hf2)
#
#
print("train")
print(train.head(10))
print("test")
print(test.head(10))
#
#
# Identify predictors and response
x = train.columns
#
# test Learning is it a bit URL TRUE/FALSE
#y = 'massiveurl'
#
# Classify and discovery for Average BitRate
#
y= 'average_bitrate_kbps'
### remove viewerID
z= 'viewerId'
x.remove(y).remove(z)
#
######## For binary classification, response should be a factor  #####
#####   train[y] = train[y].asfactor()
#####   test[y] = test[y].asfactor()
#
# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# Balance Classes to compensate unbalanced data
# Run AutoML for 25 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=25, seed=19, exclude_algos=["DRF","GLM"])
aml.train(x=x, y=y, training_frame=train)
#
sc.stop()
#
print("Model Training Done!")
#

Exception: Java gateway process exited before sending its port number